In [5]:
# !jupyter kernelspec list
%load_ext rpy2.ipython

/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex
/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:34: UserWarning: pandas >= 1.0 is not supported.
  warnings.warn('pandas >= 1.0 is not supported.')


In [12]:
# SEM
import numpy as np
import pandas as pd

In [3]:
!pip install semopy

  Created wheel for semopy: filename=semopy-1.3.7-cp36-none-any.whl size=45323 sha256=88db675f53541dc010d0c8f26a26927f23240494f865bce716bff3a05ea89241
  Stored in directory: /root/.cache/pip/wheels/fe/aa/45/35015037c1c2d6705d52154c4485d484449cf23b7e415e10db
Successfully built semopy


In [5]:
from semopy import Model

In [16]:
# The first step build a model
""" 
~は因果関係(左辺←右辺)を意味する
~~は各変数の分散
=~は潜在変数
"""
mod = """ x1 ~ x2 + x3
          x3 ~ x2 + eta1
          eta1 =~ y1 + y2 + y3
          eta1 ~ x1
      """
model = Model(mod)

In [19]:
# The second step - read a dataset
data = pd.DataFrame(index = [i for i in range(10)],columns = ['x1','x2','x3','y1','y2','y3'],data = np.random.randn(10,6))
model.load_dataset(data)

In [20]:
# The third step fit the model
from semopy import Optimizer
opt = Optimizer(model)
objective_function_value = opt.optimize()

In [21]:
from semopy.full_model import FullModel

In [22]:
from semopy.inspector import inspect
result = inspect(opt)
result

,lval,op,rval,Value,SE,Z-score,P-value
5,eta1,=~,y2,0.014301,0.253294,0.056461,0.954975
6,eta1,=~,y3,0.144477,0.486788,0.296797,0.766622
0,eta1,~,x1,0.581013,0.612611,0.948419,0.342916
2,x1,~,x2,-0.872694,0.297681,-2.931642,0.003372
1,x1,~,x3,0.850831,0.227401,3.741539,0.000183
3,x3,~,eta1,-0.522378,1.726538,-0.302558,0.762227
4,x3,~,x2,-0.791212,1.377588,-0.574346,0.565734
7,eta1,~~,eta1,1.027668,2.530649,0.406089,0.684677
8,x1,~~,x1,0.133350,0.061404,2.171679,0.029880
9,x3,~~,x3,0.634811,0.388136,1.635538,0.101936


In [23]:
# 評価指標による評価
import semopy.stats
# pd.DataFrame(ParameterStatistics(result['Value'],result['SE'],result['Z-score'],result['P-value']))
agfi = semopy.stats.calc_agfi(opt)
aic = semopy.stats.calc_aic(opt)
cfi = semopy.stats.calc_cfi(opt)
chi2 = semopy.stats.calc_chi2(opt)
gfi = semopy.stats.calc_gfi(opt)
rmsea = semopy.stats.calc_rmsea(opt)
print('agfi:{},aic:{},cfi:{},chi2:{},gfi:{},rmsea:{}'.format(agfi,aic,cfi,chi2,gfi,rmsea))

agfi:0.1157736227546311,aic:125.31860185470318,cfi:0.6236654224527066,chi2:(15.400836660847173, 0.031191100150505635),gfi:0.5873610239521612,rmsea:0.36516655605934234


In [32]:
# 可視化モジュールはあるけど、今使えない状態になっている。